In [ ]:
%%capture

!pip install nltk conllu
import nltk
import conllu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
data_train = "/content/drive/MyDrive/Ru_Syntagrus_Dataset/ru_syntagrus-ud-train.conllu"

In [ ]:
with open(data_train, mode="r", encoding="utf-8") as data:
    # Read the file contents and assign under 'annotations'
    annotations = data.read()
    
sentences = conllu.parse(annotations)

In [ ]:
sentences[1]

TokenList<Начальник, областного, управления, связи, Семен, Еремеевич, был, человек, простой, ,, приходил, на, работу, всегда, вовремя, ,, здоровался, с, секретаршей, за, руку, и, иногда, даже, писал, в, стенгазету, заметки, под, псевдонимом, ", Муха, ", ., metadata={sent_id: "2003Anketa.xml_2", text: "Начальник областного управления связи Семен Еремеевич был человек простой, приходил на работу всегда вовремя, здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом "Муха"."}>

In [ ]:
sentences[1].metadata['text']
#sentences[1].metadata['text']


'Начальник областного управления связи Семен Еремеевич был человек простой, приходил на работу всегда вовремя, здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом "Муха".'

list

In [ ]:
len(sentences)

69630

In [ ]:
sentences[1][:]


[{'id': 1,
  'form': 'Начальник',
  'lemma': 'начальник',
  'upos': 'NOUN',
  'xpos': None,
  'feats': {'Animacy': 'Anim',
   'Case': 'Nom',
   'Gender': 'Masc',
   'Number': 'Sing'},
  'head': 8,
  'deprel': 'nsubj',
  'deps': [('nsubj', 8)],
  'misc': None},
 {'id': 2,
  'form': 'областного',
  'lemma': 'областной',
  'upos': 'ADJ',
  'xpos': None,
  'feats': {'Case': 'Gen',
   'Degree': 'Pos',
   'Gender': 'Neut',
   'Number': 'Sing'},
  'head': 3,
  'deprel': 'amod',
  'deps': [('amod', 3)],
  'misc': None},
 {'id': 3,
  'form': 'управления',
  'lemma': 'управление',
  'upos': 'NOUN',
  'xpos': None,
  'feats': {'Animacy': 'Inan',
   'Case': 'Gen',
   'Gender': 'Neut',
   'Number': 'Sing'},
  'head': 1,
  'deprel': 'nmod',
  'deps': [('nmod:gen', 1)],
  'misc': None},
 {'id': 4,
  'form': 'связи',
  'lemma': 'связь',
  'upos': 'NOUN',
  'xpos': None,
  'feats': {'Animacy': 'Inan',
   'Case': 'Gen',
   'Gender': 'Fem',
   'Number': 'Sing'},
  'head': 3,
  'deprel': 'nmod',
  'deps':

###  Prune from the UD Russian SynTagRus dataset those sentences containing low-frequency lemmas with respect to a lemma frequency list 

In [ ]:
import numpy as np
import pandas as pd
freq_df = pd.read_csv("freqrnc2011.csv", sep='\t')
freq_df.head()

,Lemma,PoS,Freq(ipm),R,D,Doc
0,а,conj,8198.0,100,97,32332
1,а,intj,19.8,99,90,757
2,а,part,6.1,59,79,128
3,а,s,2.7,59,85,160
4,аа,intj,1.5,47,80,68


In [ ]:
freq_df['Freq(ipm)'].describe()

count    52138.000000
mean        18.329207
std        286.940292
min          0.400000
25%          0.800000
50%          1.900000
75%          5.600000
max      35801.800000
Name: Freq(ipm), dtype: float64

# Approach 1

In [ ]:
frequency_list = freq_df.set_index('Lemma')['Freq(ipm)'].to_dict()

In [ ]:
# Load the UD Russian SynTagRus dataset
dataset = []
with open(data_train) as file:
    sentence = []
    for line in file:
        line = line.strip()
        if not line:
            if sentence:
                dataset.append(sentence)
                sentence = []
        else:
            if line[0] != '#':
                _, word, lemma, _, _, _, _, _, _, _ = line.split('\t')
                sentence.append((word, lemma))

In [ ]:
len(frequency_list)

51734

In [ ]:
dataset[1]

[('Начальник', 'начальник'),
 ('областного', 'областной'),
 ('управления', 'управление'),
 ('связи', 'связь'),
 ('Семен', 'Семен'),
 ('Еремеевич', 'Еремеевич'),
 ('был', 'быть'),
 ('человек', 'человек'),
 ('простой', 'простой'),
 (',', ','),
 ('приходил', 'приходить'),
 ('на', 'на'),
 ('работу', 'работа'),
 ('всегда', 'всегда'),
 ('вовремя', 'вовремя'),
 (',', ','),
 ('здоровался', 'здороваться'),
 ('с', 'с'),
 ('секретаршей', 'секретарша'),
 ('за', 'за'),
 ('руку', 'рука'),
 ('и', 'и'),
 ('иногда', 'иногда'),
 ('даже', 'даже'),
 ('писал', 'писать'),
 ('в', 'в'),
 ('стенгазету', 'стенгазета'),
 ('заметки', 'заметка'),
 ('под', 'под'),
 ('псевдонимом', 'псевдоним'),
 ('"', '"'),
 ('Муха', 'муха'),
 ('"', '"'),
 ('.', '.')]

In [ ]:
# Prune the sentences that contain low-frequency lemmas
freq_threshold = 5
pruned_dataset = []
for sentence in dataset:
    include_sentence = True
    for word, lemma in sentence:
        if lemma not in frequency_list or frequency_list[lemma] < freq_threshold:
            include_sentence = False
            #print('break')
            break
        else:
          break
        
    if include_sentence:
        #print('append')
        pruned_dataset.append(sentence)

In [ ]:
len(pruned_dataset)

55791

In [ ]:
pruned_dataset[0]

[('Анкета', 'анкета'), ('.', '.')]

In [ ]:
def construct_sentences(nested_list):
    sentences = []
    for sublist in nested_list:
        sentence = ""
        for word_tuple in sublist:
            sentence += word_tuple[0] + " "
        sentences.append(sentence.strip())
    return sentences

final_sentences = construct_sentences(pruned_dataset)

In [ ]:
len(final_sentences)

55791

In [ ]:
final_sentences[:5]

['Анкета .',
 'Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .',
 'В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .',
 'Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .',
 'Приемная была обставлена просто , но по-деловому .']

In [ ]:
import pandas as pd

def extract_top_sentences(sentences, frequency_list, tokens_values, top_n):
    """
    This function takes a list of sentences, converts them to a Pandas dataframe, and puts these sentences in the first column.
    For the second column, calculates the number of tokenizations for each sentence.
    For the third column, calculates the total frequency for each sentence with respect to the given frequency list.
    Then extracts from the dataframe those rows, which have these values for tokens: [tokens_values].
    Then extracted for each value of these tokens, the top n-ranked sentences were.
    """
    # Create a list of tuples, where each tuple consists of a sentence, number of tokens, and total frequency
    data = []
    for sentence in sentences:
        tokens = len(sentence.split())
        frequency = sum([frequency_list[word] for word in sentence.split() if word in frequency_list])
        data.append((sentence, tokens, frequency))
    
    # Convert the list of tuples to a Pandas dataframe
    df = pd.DataFrame(data, columns=['Sentence', 'Tokens', 'Frequency'])
    
    # Extract the rows which have the desired number of tokens
    extracted_rows = []
    for tokens in tokens_values:
        extracted_rows.append(df[df['Tokens'] == tokens].sort_values(by='Frequency', ascending=False).head(top_n))
    
    # Concatenate the extracted rows into a single dataframe
    result = pd.concat(extracted_rows)
    
    return result, df


# Define the values for tokens to extract
tokens_values = [10, 15, 20, 25, 30, 35]

# Define the number of top-ranked sentences to extract
top_n = 200

# Call the function to extract the top-ranked sentences
result, df = extract_top_sentences(final_sentences, frequency_list, tokens_values, top_n)


In [ ]:
result.head(10)

,Sentence,Tokens,Frequency
42017,Чтобы Украину не взяли в НАТО и в Евросоюз ?,10,116615.2
12300,"Мог иметься в виду и слесарь , и столяр .",10,103118.2
51650,Потом приходил черед фонарщиков в России и в И...,10,98558.4
49559,"Как я могу быть и там , и здесь ?",10,97297.4
41360,Да и обнаружилось это в тех авариях не сразу .,10,91397.8
30462,Однако и в Грязях нас устраивало далеко не все .,10,88476.3
31962,И не только выпивку и курение в старших классах .,10,87603.2
11934,Ведь мертвые и похищенные не в состоянии себя ...,10,87515.9
27389,Размышление и самообучение в круг ее задач не ...,10,86580.6
53782,"Электричества , разумеется , не было и в помине .",10,85325.3


In [ ]:
result.shape

(1200, 3)

In [ ]:
df.head()

,Sentence,Tokens,Frequency
0,Анкета .,2,0.0
1,Начальник областного управления связи Семен Ер...,34,103107.2
2,"В приемной его с утра ожидали посетители , - к...",34,123467.4
3,Однако стиль работы Семена Еремеевича заключал...,19,101916.8
4,"Приемная была обставлена просто , но по-делово...",8,659.4


In [ ]:
df.shape

(55791, 3)

In [ ]:
#result.to_csv('/content/pruned_dataset.csv')

# Extract sentences from the UD Russian SynTagRus dataset that exist in the CSV file, along with their features

In [ ]:

import pandas as pd 
# Read the UD Russian SynTagRus dataset in CONLU format
with open(data_train, "r") as file:
    data = conllu.parse(file.read())

# Read the CSV file into a pandas dataframe
sentences_df = pd.read_csv("pruned_dataset.csv")

# Get the list of sentences from the CSV file
sentences = sentences_df["Sentence"].tolist()

# Create a new list to store only the sentences that exist in the CSV file
filtered_data = []
for sent in data:
    # Get the text of the sentence
    text = " ".join([token["form"] for token in sent])
    if text in sentences:
        filtered_data.append(sent)


# Write the filtered sentences to a new CONLL-U file

with open('filtered_SynTagRus_dataset.conllu', 'w') as f:
      f.writelines([sentence.serialize() + "\n" for sentence in filtered_data])

In [ ]:
## Test the new dataset 
with open("filtered_SynTagRus_dataset.conllu", "r") as file:
    d = conllu.parse(file.read())


[TokenList<За, неделю, начальник, управления, собирался, позвонить, Савельеву, ,, который, занимал, теперь, такой, же, пост, ,, как, и, Семен, Еремеевич, в, соседней, области, ,, и, узнать, о, деловых, качествах, Ефимовой, ., metadata={sent_id: "2003Anketa.xml_66", text: "За неделю начальник управления собирался позвонить Савельеву, который занимал теперь такой же пост, как и Семен Еремеевич в соседней области, и узнать о деловых качествах Ефимовой."}>, TokenList<На, этот, раз, она, была, в, сильном, возбуждении, и, мяла, в, кулаке, кружевной, платочек, ., metadata={sent_id: "2003Anketa.xml_151", text: "На этот раз она была в сильном возбуждении и мяла в кулаке кружевной платочек."}>, TokenList<Ребята, ,, которые, посмелей, ,, конечно, ,, уехали, на, крышах, и, на, подножках, ,, а, я, не, могла, так, …, metadata={sent_id: "2003Anketa.xml_176", text: "Ребята, которые посмелей, конечно, уехали на крышах и на подножках, а я не могла так…"}>, TokenList<Да, и, работать, она, может, только, 

In [ ]:
len(d)

1200

In [ ]:
d[0]

TokenList<За, неделю, начальник, управления, собирался, позвонить, Савельеву, ,, который, занимал, теперь, такой, же, пост, ,, как, и, Семен, Еремеевич, в, соседней, области, ,, и, узнать, о, деловых, качествах, Ефимовой, ., metadata={sent_id: "2003Anketa.xml_66", text: "За неделю начальник управления собирался позвонить Савельеву, который занимал теперь такой же пост, как и Семен Еремеевич в соседней области, и узнать о деловых качествах Ефимовой."}>

In [ ]:
d[0].metadata['text']

'За неделю начальник управления собирался позвонить Савельеву, который занимал теперь такой же пост, как и Семен Еремеевич в соседней области, и узнать о деловых качествах Ефимовой.'

In [ ]:
d[0][:]

[{'id': 1,
  'form': 'За',
  'lemma': 'за',
  'upos': 'ADP',
  'xpos': None,
  'feats': None,
  'head': 2,
  'deprel': 'case',
  'deps': [('case', 2)],
  'misc': None},
 {'id': 2,
  'form': 'неделю',
  'lemma': 'неделя',
  'upos': 'NOUN',
  'xpos': None,
  'feats': {'Animacy': 'Inan',
   'Case': 'Acc',
   'Gender': 'Fem',
   'Number': 'Sing'},
  'head': 5,
  'deprel': 'obl',
  'deps': [('obl:за:acc', 5)],
  'misc': None},
 {'id': 3,
  'form': 'начальник',
  'lemma': 'начальник',
  'upos': 'NOUN',
  'xpos': None,
  'feats': {'Animacy': 'Anim',
   'Case': 'Nom',
   'Gender': 'Masc',
   'Number': 'Sing'},
  'head': 5,
  'deprel': 'nsubj',
  'deps': [('nsubj', 5), ('nsubj', 6), ('nsubj', 25)],
  'misc': None},
 {'id': 4,
  'form': 'управления',
  'lemma': 'управление',
  'upos': 'NOUN',
  'xpos': None,
  'feats': {'Animacy': 'Inan',
   'Case': 'Gen',
   'Gender': 'Neut',
   'Number': 'Sing'},
  'head': 3,
  'deprel': 'nmod',
  'deps': [('nmod:gen', 3)],
  'misc': None},
 {'id': 5,
  'form'

# Find the match sentences between csv and conllu

In [ ]:
import pandas as pd
pruned_df = pd.read_csv("pruned_dataset.csv")

with open("filtered_SynTagRus_dataset.conllu", "r") as file:
    filtered_data = conllu.parse(file.read())

num_match_sent = 0
for sent in filtered_data:

    text = " ".join([token["form"] for token in sent])

    selected_sent = pruned_df[pruned_df["Sentence"] == text]
    if selected_sent.empty:
        print("The sentence was not found in the csv file.")
    else:
            print("Match found:")
            print("CSV sentence:", selected_sent["Sentence"])
            print("CoNLL-U sentence:", text)
            print()
            num_match_sent += 1
print(num_match_sent)

Streaming output truncated to the last 5000 lines.
CSV sentence: 676    Хотя в паспорте и записано , что выработка эне...
Name: Sentence, dtype: object
CoNLL-U sentence: Хотя в паспорте и записано , что выработка энергии начинается при ветре 3 метра в секунду , реально это происходит только с 5 м/с .

Match found:
CSV sentence: 723    Она вся - продукт российского производства и р...
Name: Sentence, dtype: object
CoNLL-U sentence: Она вся - продукт российского производства и российских технологий , что позволило выйти на новый уровень не только технических решений , но и качества .

Match found:
CSV sentence: 471    В следующем послании Онищенко постарается учес...
Name: Sentence, dtype: object
CoNLL-U sentence: В следующем послании Онищенко постарается учесть проколы и на агентурные данные уже не ссылается : анализ и только анализ .

Match found:
CSV sentence: 596    Да ведь и на украинском фронте то же самое , х...
Name: Sentence, dtype: object
CoNLL-U sentence: Да ведь и на украинск

In [ ]:
num_match_sent

1200

# match a specific sentence from conllu file with a sentence from a CSV file.

In [ ]:

import pandas as pd
pruned_df = pd.read_csv("pruned_dataset.csv")

with open("filtered_SynTagRus_dataset.conllu", "r") as file:
    filtered_data = conllu.parse(file.read())

sentence = "Чтобы Украину не взяли в НАТО и в Евросоюз ?"
selected_sent = pruned_df[pruned_df["Sentence"] == sentence]

for sent in filtered_data:

    text = " ".join([token["form"] for token in sent])

    if selected_sent["Sentence"][0] == text:
            print("Match found:")
            print("CSV sentence:", selected_sent["Sentence"])
            print("CoNLL-U sentence:", text)
        
    else:
            pass





Match found:
CSV sentence: 0    Чтобы Украину не взяли в НАТО и в Евросоюз ?
Name: Sentence, dtype: object
CoNLL-U sentence: Чтобы Украину не взяли в НАТО и в Евросоюз ?


# Check the order of sentences between csv file and conllu

In [ ]:
import pandas as pd

# Read the csv file into a pandas dataframe
df = pd.read_csv("pruned_dataset.csv")

# Read the CoNLL-U file into a list of sentences
with open("filtered_SynTagRus_dataset.conllu", "r") as f:
    conllu_sentences = f.read().split("\n\n")

# Initialize a counter
counter = 0

# Loop through each sentence in the csv dataframe
for i, row in df.iterrows():
    csv_sentence = row["Sentence"]
    # Check if the current sentence in the csv file matches the corresponding sentence in the CoNLL-U file
    if csv_sentence == conllu_sentences[counter]:
        print("Match found:")
        print("CSV sentence:", csv_sentence)
        print("CoNLL-U sentence:", conllu_sentences[counter])
        counter += 1
    else:
        print("ERROR: Order of sentences is different in csv file and CoNLL-U file.")
        break


ERROR: Order of sentences is different in csv file and CoNLL-U file.
